In [1]:
import glob
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import rocks

rocks.set_log_level("error")
import time as t


from astropy.time import Time
import requests
import io

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation

from scipy.signal import argrelextrema

import ssptools

In [3]:
import figure_setup as fs

# Get ZTF fit

In [4]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [5]:
# Threshold for selection (of non-zero values)
thres = 1e-3

# Minimum phase angle to consider
min_phase = 3

In [6]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

S_color = {"g": -0.3928, "r": 0.2913}
sun_color = {"g": -0.3044, "r": 0.1903}

color_C = -(sun_color["g"] - sun_color["r"])
color_S = -(S_color["g"] - S_color["r"])

fink_colors = ["#15284F", "#F5622E"]

V_minus_g = -0.2833
V_minus_r = 0.1777

In [7]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [8]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [9]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "colors.g-i.color.value",
    "colors.g-i.color.error.min",
    "colors.g-i.color.error.max",
    "colors.g-i.facility",
    "colors.g-i.observer",
    "colors.g-i.epoch",
    "colors.g-i.delta_time",
    "colors.g-i.id_filter_1",
    "colors.g-i.id_filter_2",
    "colors.g-i.phot_sys",
    "colors.g-i.technique",
    "colors.i-z.color.value",
    "colors.i-z.color.error.min",
    "colors.i-z.color.error.max",
    "colors.i-z.facility",
    "colors.i-z.observer",
    "colors.i-z.epoch",
    "colors.i-z.delta_time",
    "colors.i-z.id_filter_1",
    "colors.i-z.id_filter_2",
    "colors.i-z.phot_sys",
    "colors.i-z.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)

In [10]:
data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [11]:
# data

In [12]:
thres = 1e-3

# HG Parameeter
mask_HG_g = (data.HG_H_g.notna()) & (data.HG_G_g.notna())
mask_HG_r = (data.HG_H_r.notna()) & (data.HG_G_r.notna())
mask_HG_fit = (data.HG_fit == 0) & (data.HG_status >= 2)
mask_HG = mask_HG_g & mask_HG_r & mask_HG_fit

# HG1G2 parameters
mask_HG1G2_g = (
    (data.HG1G2_G1_g > thres)
    & (data.HG1G2_G2_g > thres)
    & ((1 - data.HG1G2_G1_g - data.HG1G2_G2_g) > thres)
)
mask_HG1G2_r = (
    (data.HG1G2_G1_r > thres)
    & (data.HG1G2_G2_r > thres)
    & ((1 - data.HG1G2_G1_r - data.HG1G2_G2_r) > thres)
)
mask_HG1G2_fit = (data.HG1G2_fit == 0) & (data.HG1G2_status >= 2)
mask_HG1G2 = mask_HG1G2_fit & mask_HG1G2_g & mask_HG1G2_r

# SHG1G2 ZTF
mask_SHG1G2_g = (
    (data.SHG1G2_G1_g > thres)
    & (data.SHG1G2_G2_g > thres)
    & ((1 - data.SHG1G2_G1_g - data.SHG1G2_G2_g) > thres)
)
mask_SHG1G2_r = (
    (data.SHG1G2_G1_r > thres)
    & (data.SHG1G2_G2_r > thres)
    & ((1 - data.SHG1G2_G1_r - data.SHG1G2_G2_r) > thres)
)
mask_SHG1G2_ZTF = mask_SHG1G2_g & mask_SHG1G2_r
mask_SHG1G2_fit = (data.SHG1G2_fit == 0) & (data.SHG1G2_status >= 2)
mask_SHG1G2 = mask_SHG1G2_fit & mask_SHG1G2_ZTF


# Spin solution suspicous: RA=={0,180,360}, DEC==0
maskSpin = (
    (data.SHG1G2_alpha0 > thres)
    & (np.abs(360 - data.SHG1G2_alpha0) > thres)
    & (np.abs(data.SHG1G2_alpha0 - 180) > thres)
    & (np.abs(data.SHG1G2_delta0) > thres)
)

# Phase coverage
maskPhase = data.min_phase < 2.5

# Global mask
maskFINK = mask_SHG1G2 & maskSpin  # & maskPhase


print(f" All data       : {len(data):6d}  ({100:>6.2f}%)")
print()
print(
    f"  Mask HG g      : {len(data[mask_HG_g]):6d}  ({100.*len(data[mask_HG_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG r      : {len(data[mask_HG_r]):6d}  ({100.*len(data[mask_HG_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG g+r    : {len(data[mask_HG]):6d}  ({100.*len(data[mask_HG])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask HG1G2 g   : {len(data[mask_HG1G2_g]):6d}  ({100.*len(data[mask_HG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 r   : {len(data[mask_HG1G2_r]):6d}  ({100.*len(data[mask_HG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 g+r : {len(data[mask_HG1G2]):6d}  ({100.*len(data[mask_HG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask SHG1G2 g  : {len(data[mask_SHG1G2_g]):6d}  ({100.*len(data[mask_SHG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 r  : {len(data[mask_SHG1G2_r]):6d}  ({100.*len(data[mask_SHG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 g+r: {len(data[mask_SHG1G2]):6d}  ({100.*len(data[mask_SHG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask Spin      : {len(data[maskSpin]):6d}  ({100.*len(data[maskSpin])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask FINK      : {len(data[maskFINK]):6d}  ({100.*len(data[maskFINK])/len(data):>6.2f}%)"
)
for minphase in [2, 3, 4, 5]:
    maskPhase = data.min_phase < minphase
    print(
        f"  Mask phase {minphase}   : {len(data[maskPhase]):6d}  ({100.*len(data[maskPhase])/len(data):>6.2f}%)"
    )

 All data       : 122675  (100.00%)

  Mask HG g      : 122371  ( 99.75%)
  Mask HG r      : 122373  ( 99.75%)
  Mask HG g+r    : 120010  ( 97.83%)

  Mask HG1G2 g   :  63922  ( 52.11%)
  Mask HG1G2 r   :  68956  ( 56.21%)
  Mask HG1G2 g+r :  47175  ( 38.46%)

  Mask SHG1G2 g  :  79912  ( 65.14%)
  Mask SHG1G2 r  :  84779  ( 69.11%)
  Mask SHG1G2 g+r:  64209  ( 52.34%)

  Mask Spin      : 118811  ( 96.85%)

  Mask FINK      :  62943  ( 51.31%)
  Mask phase 2   :  62923  ( 51.29%)
  Mask phase 3   :  86161  ( 70.24%)
  Mask phase 4   : 100699  ( 82.09%)
  Mask phase 5   : 109011  ( 88.86%)


# General description of the sample

In [13]:
data.loc[maskFINK, "sso_class"].value_counts()

sso_class
MB>Inner        20645
MB>Middle       19920
MB>Outer        17200
Phocaea           581
Trojan            544
Hungaria          452
Mars-Crosser      439
MB>Cybele         276
MB>Hilda          243
NEA>Amor           44
NEA>Apollo         44
NEA>Aten            3
Centaur             2
KBO>SDO             1
Name: count, dtype: int64

In [14]:
As = data.loc[maskFINK, "orbital_elements.semi_major_axis.value"]
Es = data.loc[maskFINK, "orbital_elements.eccentricity.value"]
node = data.loc[maskFINK, "orbital_elements.inclination.value"]

fig, ax = plt.subplots(figsize=(12, 8))

ax.scatter(As, np.sin(np.radians(node)), marker=".", alpha=0.2, s=2)


ax.set_xlim(1.5, 5.5)
ax.set_ylim(0, 0.6)
ax.set_xlabel("Semi-major axis (au)")
ax.set_ylabel(r"Sine of inclination ($\sin$i)")

# fig.savefig(f'{data_fink}plots/types.png', facecolor='white', dpi=150)

Text(0, 0.5, 'Sine of inclination ($\\sin$i)')

# Spins!

In [15]:
data.columns[data.columns.str.contains("alpha0")]

Index(['SHG1G2_alpha0', 'SHG1G2_dalpha0', 'SHG1G2_alpha0_alt'], dtype='object')

In [16]:
# Create alternative spins solutions
xax = "SHG1G2_alpha0"
yax = "SHG1G2_delta0"

# Already done at Fink level
# data["SHG1G2_alpha0_alt"] = (data["SHG1G2_alpha0"] + 180) % 360
# data["SHG1G2_delta0_alt"] = -data["SHG1G2_delta0"]


data["SHG1G2_alpha0_rand"] = data["SHG1G2_alpha0"]
data["SHG1G2_delta0_rand"] = data["SHG1G2_delta0"]
cond = (data.index % 2) == 0
data.loc[cond, "SHG1G2_alpha0_rand"] = data.loc[cond, "SHG1G2_alpha0_alt"]
data.loc[cond, "SHG1G2_delta0_rand"] = data.loc[cond, "SHG1G2_delta0_alt"]


# Convert to EC
coords = SkyCoord(
    ra=data["SHG1G2_alpha0"].values * u.deg,
    dec=data["SHG1G2_delta0"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon"] = coords.heliocentricmeanecliptic.lon.value
data["lat"] = coords.heliocentricmeanecliptic.lat.value

coords = SkyCoord(
    ra=data["SHG1G2_alpha0_alt"].values * u.deg,
    dec=data["SHG1G2_delta0_alt"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_alt"] = coords.heliocentricmeanecliptic.lon.value
data["lat_alt"] = coords.heliocentricmeanecliptic.lat.value


coords = SkyCoord(
    ra=data["SHG1G2_alpha0_rand"].values * u.deg,
    dec=data["SHG1G2_delta0_rand"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_rand"] = coords.heliocentricmeanecliptic.lon.value
data["lat_rand"] = coords.heliocentricmeanecliptic.lat.value

In [17]:
# Obliquity of the spin
data["lon_orbit"] = data["orbital_elements.node_longitude.value"] - 90
data["lat_orbit"] = 90.0 - data["orbital_elements.inclination.value"]
data["obliquity"] = data[["lon", "lat", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)
data["obliquity_alt"] = data[["lon_alt", "lat_alt", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)

/tmp/ipykernel_987798/3937172300.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
/tmp/ipykernel_987798/3937172300.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])


# Figure longitude vs inclination

In [78]:
Is = np.sin(np.deg2rad(data["orbital_elements.inclination.value"]))

# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    2,
    5,
    figsize=fs.figsize(1, aspect=3),
    # sharex=True,
    # sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "left": 0.05,
        "right": 0.98,
        "top": 0.98,
        "bottom": 0.13,
    },
)
m = maskFINK

# --------------------------------------------------------------------------------
# Distributions
step = 0.04
count = 0
for i in range(0, 2):
    for j in range(0, 5):
        mi = (Is[m] >= count * step) * (Is[m] < (count + 1) * step)
        ax[i, j].hist(data["lon_rand"][m][mi], bins=45, density=False, histtype="step")

        # ax[i, j].text(
        #     180,
        #     350,
        #     r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
        #     ha="center",
        #     fontsize="x-small",
        # )
        # ax[i, j].text(
        #     180,
        #     50,
        #     "{:d} ({:.2f}%)".format(len(data["lon_rand"][m][mi]),
        #         len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
        #     ),
        #     ha="center",
        #     fontsize="x-small",
        # )

        if i==0:
            yTit, yNum, yFrac = 40, 350, 350
        else:
            yTit, yNum, yFrac = 170, 40, 40           
        
        ax[i, j].text(
            180,
            yTit,
            r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
            ha="center",
            fontsize="x-small",
        )
        ax[i, j].text(
            20,
            yNum,
            "{:,d}".format(len(data["lon_rand"][m][mi])),
            ha="left",
            fontsize="x-small",
        )
        ax[i, j].text(
            340,
            yFrac,
            "({:.2f}%)".format(
                len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
            ),
            ha="right",
            fontsize="x-small",
        )

        count += 1

        # only on the last one
        # ax[i, j].legend(fontsize=10)

# --------------------------------------------------------------------------------
# Axes
ax[1, 2].set_xlabel(r"Longitude of the spin ($^{\circ}$)")

for a in ax[:, 0]:
    a.set_ylabel(r"Count")

for a in ax[0, :].ravel():
    a.set_yticks([0,100,200,300,400])
for a in ax[1, :].ravel():
    a.set_yticks([0,50,100,150,200])
for a in ax[:, 1:].ravel():
    a.set_yticklabels(['', '', '', '', ''])

for a in ax.ravel():
    a.set_xticks([0, 90, 180, 270, 360])
for a in ax[1, :-1]:
    a.set_xticklabels([0, 90, 180, 270, " "])
ax[1, -1].set_xticklabels([0, 90, 180, 270, 360])


ax[0, 0].set_ylim(0, 400)
ax[0, 0].set_xlim(0, 360)


# --------------------------------------------------------------------------------
fig.savefig(os.path.join(data_fink, "gfx", "article", "longitude_inclination.pgf"))
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination.png"),
    facecolor="white",
    dpi=180,
)